In [117]:
from bs4 import BeautifulSoup
import requests
import re
import unicodedata

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:73.0) Gecko/20100101 Firefox/73.0'}

In [3]:
html_soup = requests.get("https://www.practicaespanol.com/noticias/", headers=headers)
parsed_soup = BeautifulSoup(html_soup.content, 'html.parser')

In [4]:
# Get total number of pages of articles 
numberPages = parsed_soup.find('span', {"class": "pages"}).get_text()

In [5]:
# Parse this number as an int
numberPages = int(re.search(r'\w+\d$', numberPages).group())

In [9]:
# Get the number of target URLS
targetURLs = []
for page in range(1, numberPages + 1):
    targetURL = "https://www.practicaespanol.com/noticias/page/" + str(page) + "/"
    targetURLs.append(targetURL)

In [8]:
# Loop through all of the pages and get the meta data
for targetURL in targetURLs:
    html_soup = requests.get(targetURL, headers=headers)
    parsed_soup = BeautifulSoup(html_soup.content, 'html.parser')

IndentationError: unexpected indent (<ipython-input-8-4ae4df015c7b>, line 2)

In [11]:
html_soup = requests.get('https://www.practicaespanol.com/noticias/page/21/', headers=headers)
parsed_soup = BeautifulSoup(html_soup.content, 'html.parser')

In [15]:
# This corresponds to the box on each article
article = parsed_soup.find_all('article')[0]

In [21]:
# Get title
article.find('h2', {'class': 'entry-title'}).get_text()

'El pueblo de Suiza que una vez cada 25 años se regala una fiesta vitivinícola de unos 100 millones de francos'

In [23]:
# Get category
article.find('span', {'class': 'categoria2'}).get_text()

'Viajes '

In [34]:
# Get level
level = [img['alt'] for img in article.find_all('img', alt=True) if 'Nivel' in img['alt']]
level = level[0][-2:]

'C1'

In [55]:
# Get link
link = article.find('h2', {'class': 'entry-title'}).findChild("a" , recursive=False, href=True)
link = link['href']

'https://www.practicaespanol.com/el-pueblo-de-suiza-que-una-vez-cada-25-anos-se-regala-una-fiesta-del-vino-de-unos-100-millones-de-francos/'

In [174]:
article_text = 'https://www.practicaespanol.com/los-refugiados-venezolanos-que-en-brasil-disputan-la-comida-a-buitres-y-perros/'
html_soup = requests.get(article_text, headers=headers)
parsed_soup = BeautifulSoup(html_soup.content, 'html.parser')

In [175]:
# Remove HTML page
try:
    for script in parsed_soup('script'):
        script.decompose()
except:
    pass

In [176]:
# Get date

In [177]:
coloredSpans = parsed_soup.find_all('span', attrs={'style': 'color: #993366;'})
targetColorSan = [span.get_text() for span in coloredSpans if 'EFE/PracticaEspañol' in span.get_text()]
date = targetColorSan[0].split(',')[0]
date

'18 de febrero de 2020'

In [178]:
# Get resumen
resumen = parsed_soup.find('h2').get_text()
resumen = unicodedata.normalize("NFKD", resumen)
resumen

'Joedson Alves / Pacaraima (Brasil) /  Un grupo de refugiados venezolanos montó una pequeña villa en el basurero municipal de Pacaraima, la ciudad brasileña en la frontera con Venezuela, en donde prácticamente disputa restos de comida con buitres y perros.'

In [179]:
# Get article text
# Get all the ps in the aticle conent
textChunks = parsed_soup.find('div', {'class': 'entry-content'}).findChildren("p")

In [180]:
article_text = " ".join([chunk.get_text() for chunk in textChunks]).replace("\n", "")

In [181]:
article_text = unicodedata.normalize("NFKD", article_text).strip()

In [182]:
article_text

'En unas seis pequeñas viviendas fabricadas con material de desecho y escondidas en un pequeño bosque en medio del basurero de Pacaraima, viven unos cuarenta refugiados venezolanos, según pudo constatar Efe en una visita al lugar, ubicado a unos 10 kilómetros de la frontera entre ambos países. Pacaraima, el único paso en los casi 2.200 kilómetros de frontera terrestre entre Brasil y Venezuela, recibe diariamente unos 400 venezolanos que huyen de la crisis económica, política, social y humanitaria de su país. Y muchos, sin recursos para continuar hacia Boa Vista, la capital del estado de Roraima, terminan estableciéndose en Pacaraima y sus alrededores, para vivir de lo que pueden, lo que ha generado tensión por el aumento de la violencia en la región y ha provocado hasta ataques xenófobos. Pese a que aseguran que se dedican a rebuscar entre la basura latas, metales, cartones y otros materiales reciclables que pueden ser revendidos, los venezolanos establecidos en el basure